### Northwestern Memorial Pricing Data

### Description
Consolidate charge and rates data from the following providers.

Target URL: `https://www.nm.org/patients-and-visitors/billing-and-insurance/chargemaster`

Providers
* Northwestern Memorial Hospital
* Northwestern Medicine Central DuPage Hospital
* Northwestern Medicine Lake Forest Hospital
* Northwestern Medicine Delnor Hospital 
* Northwestern Medicine Kishwaukee Hospital
* Northwestern Medicine Valley West Hospital
* Northwestern Medicine Marianjoy Rehabilitation Hospital
* Northwestern Medicine Huntley Hospital
* Northwestern Medicine McHenry Hospital
* Northwestern Medicine Woodstock Hospital
* Northwestern Medicine Palos Hospital

Parsed
* Excel charge files for all but Palos. Sections: Services, Medications, Surgical Supplies.
* Negotiated rate files for all but Palos

TODO
* Palos custom Excel negotiate rate file. Charge file N/A.

Output data sets (dataframes)
* charge data
    * `services`
    * `medication`
    * `surgical_supplies`
* negotiated rate data
    * `rates`
    
Notes
* There are .txt files in page source that are not visible on the main page. 
    * The charges in .txt do not match .xlsx files. They appear to be from different time periods.
    * These .txt files appear to contain negotiated rate data while the .xlsx files are less useful charge data.

In [ ]:
from pprint import pprint
import re

from bs4 import BeautifulSoup
import pandas as pd
import requests

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)

url = 'https://www.nm.org/patients-and-visitors/billing-and-insurance/chargemaster'
soup = BeautifulSoup(requests.get(url).text, 'html.parser')

### gather charge URLs
charge_elements = soup.select('.panel-content')[3].find_all('a')
charge_lookup = {}
for e in charge_elements:
    # TODO: make edge case for Palos non-standardized file
    if 'Palos Hospital' in e.text:
        continue
    charge_lookup[e.text] = url + e['href']
    
pprint(charge_lookup)

### gather negotiated rate URLs
rate_elements = soup.select('.panel-content')[1].find_all('a')
rate_lookup = {}
for e in rate_elements:
    if 'Palos Hospital' in e.text:
        # TODO: make edge case for Palos non-standardized file
        continue
    rate_lookup[e.text] = url + e['href']
    
print()
pprint(rate_lookup)

### Retrieve charge data into pandas

In [ ]:
#############################################
### retrieve and consolidate charges
#############################################

def cleanup_cols(df):
    """We will pipe this func on each dataframe in loop before consolidating df's."""
    df.columns = [col.lower().replace(' ', '_') for col in df.columns]
    return df

services = pd.DataFrame()
medications = pd.DataFrame()
surgical_supplies = pd.DataFrame()

for label, url in charge_lookup.items():
    print(url)
    _services = (pd.read_excel(url, sheet_name='Services')
                   .assign(source=label)
                   .pipe(cleanup_cols)
                   .rename(columns={'billing_code': 'charge_code'})
                )
    
    _medications = (pd.read_excel(url, sheet_name='Medications')
                      .assign(source=label)
                      .pipe(cleanup_cols)
                      .rename(columns={'code_id': 'supply_code', 'code_description': 'code_name'})
                   )
    
    # concatenate with consolidated df's
    services = pd.concat([services, _services])
    medications = pd.concat([medications, _medications])
    
    # Marianjoy and Woodstock do not have Surgical Supplies tab (2/14/22)
    try:
        _surgical_supplies = (pd.read_excel(url, sheet_name='Surgical Supplies')
                                .assign(source=label)
                                .pipe(cleanup_cols)
                                .rename(columns={'code_id': 'supply_code', 'code_description': 'code_name'})
                             )
        
        surgical_supplies = pd.concat([surgical_supplies, _surgical_supplies])
    except ValueError as e:
        print('\t', e)
        
del _services
del _medications
del _surgical_supplies

In [ ]:
#############################################
### retrieve and consolidate negotiated rates
#############################################
rates = pd.DataFrame()
for label, url in rate_lookup.items():
    print(url)
    _rates = (pd.read_csv(url, encoding='latin1', sep='|')
                .assign(source=label)
                .pipe(cleanup_cols)
             )
    
    charge_cols = [col for col in _rates.columns if re.search('discount|charge', col) is not None]
    non_charge_cols = set(_rates.columns) - set(charge_cols)    
    
    _rates = _rates.melt(id_vars=non_charge_cols, value_vars=charge_cols, var_name='rate_amt_type', value_name='rate_amt')
    rates = pd.concat([rates, _rates])
    
rates['rate_amt'] = rates['rate_amt'].str.replace('$', '', regex=False).astype(float)

### Inspect charge data
See `source` column for provider source.

In [ ]:
services.head()

In [ ]:
medications.head()

In [ ]:
surgical_supplies.head()

### Inspect rates data
See `source` column for provider source.

In [ ]:
# query for DRG 795 Normal Newborn rates
(rates
    .query("service_description.str.contains('Normal Newborn')")
    .sort_values(by=['source', 'rate_amt_type'])
    .head(20)
)

Explore billing codes and descriptions

In [ ]:
(rates.query("billing_code.str.contains('7373') | \
             service_description.str.lower().str.contains('xray')")
      .loc[:, ['billing_code', 'service_description']]
      .drop_duplicates()
      .sort_values(by=['billing_code'])
)

In [ ]:
(rates.query("service_description.str.lower().str.contains('delivery')")
      .loc[:, ['billing_code', 'service_description']]
      .drop_duplicates()
      .sort_values(by=['billing_code'])
)

In [ ]:
(rates.query("billing_code.str.contains('45378') | \
             service_description.str.lower().str.contains('diagnostic colonoscopy')")
      .loc[:, ['billing_code', 'service_description']]
      .drop_duplicates()
      .sort_values(by=['billing_code'])
)

### Ad hoc queries, benchmarking
* Query for distributions of negotiated rates across providers. 
* Exclude min/max, self-pay, and charge amounts.

In [ ]:
rates.columns

In [ ]:
# code = '73630'
# code = '795'
code =  '59409'

g = (rates
    .query("billing_code.str.contains(@code)")
    .query("rate_amt_type.str.contains(r'^negotiated_charge.*')")
    .loc[:, ['source', 'rate_amt']]
    .groupby(['source'], as_index=True)
    .describe()
    .loc[:, 'rate_amt']
    .sort_values(by=['mean'], ascending=False)
    .applymap(lambda x: '{:,.0f}'.format(x))
)

print(f"Benchmark pricing for code: {code}")
print(g.to_markdown())

In [ ]:
(rates
  .query("billing_code.str.contains('73630', regex=False)")
  .query("source=='Northwestern Memorial Hospital'") 
  #.query("rate_amt_type.str.contains('cash')")
  .query("rate_amt_type.str.contains('negotiated_charge:_bcbs')")
  .sort_values(by=['rate_amt'], ascending=[True])
)

# Rush University Medical Center

In [ ]:
def tweak_df(df, facility_type=None, label=None): 
    """Skips first row."""
    if facility_type=='OP' or label=='Rush University Medical Center':
        # same effect as skipping first row
        df.columns = df.iloc[0, :]
        df = df.drop(index=0).reset_index(drop=True)
    df.columns = [str(col).lower().replace(' ', '_') for col in df.columns]    
    return df

rush_lookup = {
    "Rush University Medical Center": "https://www.rush.edu/sites/default/files/media-documents/rumc-standard-charges-2022.xlsx",
    "Rush Copley Medical Center": "https://www.rush.edu/sites/default/files/media-documents/rcmc-standard-charges-2022.xlsx",
    "Rush Oak Park Hospital": "https://www.rush.edu/sites/default/files/media-documents/roph-standard-charges-2022.xlsx",
}

df_ip = pd.DataFrame()
df_op = pd.DataFrame()

for label, url in rush_lookup.items():
    _df_ip = (pd.read_excel(url, sheet_name='IP')
                .pipe(tweak_df, facility_type='IP', label=label)
                .assign(source=label)
             )

    _df_op = (pd.read_excel(url, sheet_name='OP')
                .pipe(tweak_df, facility_type='OP')
                .dropna(subset=['code_id'])
                .assign(source=label)
             )
    
    df_ip = pd.concat([df_ip, _df_ip])
    df_op = pd.concat([df_op, _df_op])

In [ ]:
df_ip = df_ip.rename(columns={'drg_code': 'code'})

In [ ]:
df_op.shape

In [ ]:
df_ip.query("drg_code=='MS795'").T

In [ ]:
# df_op.head()

In [ ]:
# df_op.query("~ndc.isna()").head(10)

In [ ]:
non_rate_cols = ['code_type', 'code_id', 'code_cpt/hcpcs', 'ndc', 'code_description', 'source']
rate_cols = set(df_op.columns) - set(non_rate_cols)
melted = df_op.melt(id_vars=non_rate_cols, value_vars=rate_cols, var_name='rate_amt_type', value_name='rate_amt')

In [ ]:
melted = melted.rename({"code_cpt/hcpcs": "code"})

In [ ]:
# melted.shape

In [ ]:
melted.head()